In [1]:
from javascript import require, On, Once, AsyncTask, once, off, globalThis
import json
import logging
import time
import vision

import building

MINECRAFT_VERSION = '1.20.2'

craft = require('./control_primitives/craftItem.js')
globalThis.craft = craft
explore = require('./control_primitives/exploreUntil.js')
globalThis.explore = explore
move = require('./control_primitives/goToPosition.js')
globalThis.move = move
kill = require('./control_primitives/killMob.js')
globalThis.kill = kill
collectPosition = require('./control_primitives/mineBlockPosition.js')
globalThis.collectPosition = collectPosition
collectType = require('./control_primitives/mineBlockType.js')
globalThis.collectType = collectType
pickupDroppedItem = require('./control_primitives/pickupDroppedItem.js')
globalThis.pickupDroppedItem = pickupDroppedItem
place = require('./control_primitives/placeItem.js')
globalThis.place = place
shoot = require('./control_primitives/shoot.js')
globalThis.shoot = shoot
smelt = require('./control_primitives/smeltItem.js')
globalThis.smelt = smelt
chest = require('./control_primitives/useChest.js')
globalThis.chest = chest

Vec3 = require('vec3').Vec3
globalThis.Vec3 = Vec3
mineflayer = require('mineflayer')
globalThis.mineflayer = mineflayer
pathfinder = require('mineflayer-pathfinder')
globalThis.pathfinder = pathfinder
pvp = require('mineflayer-pvp').plugin
globalThis.pvp = pvp
mcData = require('minecraft-data')(MINECRAFT_VERSION)
globalThis.mcData = mcData
armorManager = require("mineflayer-armor-manager")
globalThis.armorManager = armorManager
autoeat = require('mineflayer-auto-eat').plugin
globalThis.autoeat = autoeat
collectblock = require('mineflayer-collectblock').plugin
globalThis.collectblock = collectblock
hawkeye = require('minecrafthawkeye')
globalThis.hawkeye = hawkeye
toolPlugin = require('mineflayer-tool').plugin
globalThis.toolPlugin = toolPlugin
# mineflayerViewer = require('prismarine-viewer')

RANGE_GOAL = 1

# Fixing some crafting recipes involving planks
all_planks = [name for name in mcData.blocksByName if 'planks' in name]
crafting_table_list = []
bookshelves_list = []

stick_list = [] # [{ 'inShape': [int(mcData.itemsByName['bamboo']['id']), int(mcData.itemsByName['bamboo']['id'])], 'result': { 'id': 807, 'count': 1 } }]

shield_list = []
wood_sword_list = []
wood_pickaxe_list = []
wood_axe_list = []
wood_shovel_list = []
wood_hoe_list = []

for name in all_planks:
    crafting_table_list.append({ 'inShape': [[int(mcData.itemsByName[name]['id']), int(mcData.itemsByName[name]['id'])], [int(mcData.itemsByName[name]['id']), int(mcData.itemsByName[name]['id'])]], 'result': { 'id': 278, 'count': 1 } })
    bookshelves_list.append({ 'inShape': [[int(mcData.itemsByName[name]['id']), int(mcData.itemsByName[name]['id']), int(mcData.itemsByName[name]['id'])], [int(mcData.itemsByName['book']['id']), int(mcData.itemsByName['book']['id']), int(mcData.itemsByName['book']['id'])], [int(mcData.itemsByName[name]['id']), int(mcData.itemsByName[name]['id']), int(mcData.itemsByName[name]['id'])]], 'result': { 'id': 264, 'count': 1 } })
    
    stick_list.append({ 'inShape': [[int(mcData.itemsByName[name]['id'])], [int(mcData.itemsByName[name]['id'])]], 'result': { 'id': 807, 'count': 4 } })
    
    shield_list.append({ 'inShape': [[int(mcData.itemsByName[name]['id']), int(mcData.itemsByName['iron_ingot']['id']), int(mcData.itemsByName[name]['id'])], [int(mcData.itemsByName[name]['id']), int(mcData.itemsByName[name]['id']), int(mcData.itemsByName[name]['id'])], [None, int(mcData.itemsByName[name]['id']), None]], 'result': { 'id': 1116, 'count': 1 } })
    wood_sword_list.append({ 'inShape': [[int(mcData.itemsByName[name]['id'])], [int(mcData.itemsByName[name]['id'])], [int(mcData.itemsByName['stick']['id'])]], 'result': { 'id': 777, 'count': 1 } })
    wood_pickaxe_list.append({ 'inShape': [[int(mcData.itemsByName[name]['id']), int(mcData.itemsByName[name]['id']), int(mcData.itemsByName[name]['id'])], [None, int(mcData.itemsByName['stick']['id']), None], [None, int(mcData.itemsByName['stick']['id']), None]], 'result': { 'id': 779, 'count': 1 } })
    wood_axe_list.append({ 'inShape': [[int(mcData.itemsByName[name]['id']), int(mcData.itemsByName[name]['id'])], [int(mcData.itemsByName[name]['id']), int(mcData.itemsByName['stick']['id'])], [None, int(mcData.itemsByName['stick']['id'])]], 'result': { 'id': 780, 'count': 1 } })
    wood_shovel_list.append({ 'inShape': [[int(mcData.itemsByName[name]['id'])], [int(mcData.itemsByName['stick']['id'])], [int(mcData.itemsByName['stick']['id'])]], 'result': { 'id': 778, 'count': 1 } })
    wood_hoe_list.append({ 'inShape': [[int(mcData.itemsByName[name]['id']), int(mcData.itemsByName[name]['id'])], [None, int(mcData.itemsByName['stick']['id'])], [None, int(mcData.itemsByName['stick']['id'])]], 'result': { 'id': 781, 'count': 1 } })

mcData.recipes[mcData.itemsByName['crafting_table']['id']] = crafting_table_list
mcData.recipes[mcData.itemsByName['bookshelf']['id']] = bookshelves_list
mcData.recipes[mcData.itemsByName['stick']['id']] = stick_list
mcData.recipes[mcData.itemsByName['shield']['id']] = shield_list
mcData.recipes[mcData.itemsByName['wooden_sword']['id']] = wood_sword_list
mcData.recipes[mcData.itemsByName['wooden_pickaxe']['id']] = wood_pickaxe_list
mcData.recipes[mcData.itemsByName['wooden_axe']['id']] = wood_axe_list
mcData.recipes[mcData.itemsByName['wooden_shovel']['id']] = wood_shovel_list
mcData.recipes[mcData.itemsByName['wooden_hoe']['id']] = wood_hoe_list



# Load bot
def load_bot(username=None):
  # Create bot
  if not username:
    random_number = 176
    username = f'poo_bucket{random_number}'

  bot = mineflayer.createBot({ 
      'host': 'localhost',
      'port': 22222,
      'username': username, 
      'hideErrors': False 
  })

  once(bot, 'login')
  bot.chat('I spawned')
  #   mineflayerViewer(bot, { 'port': 3000, 'firstPerson': True })

  # Load plugins
  bot.loadPlugin(collectblock)
  bot.loadPlugin(hawkeye)
  bot.loadPlugin(pathfinder.pathfinder)
  bot.loadPlugin(pvp)
  bot.loadPlugin(toolPlugin)

  globalThis.mcData = mcData
  globalThis.pathfinder = pathfinder
  movements = pathfinder.Movements(bot, mcData)
  bot.pathfinder.setMovements(movements)

  # Auto armor
  bot.loadPlugin(armorManager)
  bot.armorManager.equipAll()

  # Auto eat
  bot.loadPlugin(autoeat)
  bot.autoEat.options.priority = 'foodPoints'
  bot.autoEat.options.startAt = 14
  bot.autoEat.options.bannedFood.push('golden_apple')

  bot.pathfinder.thinkTimeout = 5000

  
  return bot, movements

In [2]:
bot, movements = load_bot('poo_bucket176')
globalThis.bot = bot

[JSE] Mineflayer detected that you are using a deprecated event (physicTick)! Please use this event (physicsTick) instead.



[Function (anonymous)]

In [5]:
explore.exploreUntil(bot, Vec3(-1, 0, 0), 30, timeout=5)

Exception: Call to 'exploreUntil' timed out. Increase the timeout by setting the `timeout` keyword argument.

In [12]:
collectType.mineBlockType(bot, 'oak_log', 50, timeout=60000)

In [8]:
explore.exploreUntil(bot, Vec3(1, 0, 0), 30, timeout=60000)

In [18]:
craft.craftItem(bot, 'wooden_pickaxe', 1, timeout=60000)

True

In [16]:
bot.inventory

Window {
  _events: [Object: null prototype] {},
  _eventsCount: 0,
  _maxListeners: undefined,
  acceptOutsideWindowClick: [Function: acceptClick],
  acceptInventoryClick: [Function: acceptClick],
  acceptNonInventorySwapAreaClick: [Function: acceptClick],
  acceptSwapAreaLeftClick: [Function: acceptClick],
  acceptSwapAreaRightClick: [Function: acceptClick],
  acceptCraftingClick: [Function: acceptClick],
  id: 0,
  type: 'minecraft:inventory',
  title: 'Inventory',
  slots: [
    null,
    null,
    null,
    null,
    null,
    null,
    null,
    null,
    null,
    Item {
      type: 110,
      count: 10,
      metadata: 0,
      nbt: null,
      stackId: null,
      name: 'oak_log',
      displayName: 'Oak Log',
      stackSize: 64,
      maxDurability: undefined,
      slot: 9
    },
    null,
    null,
    null,
    null,
    null,
    null,
    null,
    null,
    null,
    null,
    null,
    null,
    null,
    null,
    null,
    null,
    null,
    null,
    null,
    nul

In [13]:
place.placeItem(bot, 'crafting_table', Vec3(284,79,-2), timeout=60000)

In [5]:
import env_info
import memory
import perception
import action
import goal_planning

percept = perception.Module()
mem = memory.Module()
act = action.Module()
goal = goal_planning.Module()

KeyboardInterrupt: 

In [ ]:
import agent_llms
import json
from javascript import require, globalThis
findAndParseJsonLikeText = require('json-like-parse')

agent_prompt = agent_llms.get_prompts()['select_module']
modules = json.loads(open('modules.json').read())

In [ ]:
# Here we have our main Agent loop

current_goal = ''
plan = ''
previous_action_outcomes, current_goal_progress = [], ''
previous_module_outcome = ''
while True:

    # Call agent module to get the next action
    prompt_input = {
        'current_environment': env_info.getPromptInfo(bot),
        'goal': current_goal,
        'plan': plan,
        'goal_progress': current_goal_progress,
        'previous_action_outcomes': previous_action_outcomes,
        'modules': modules,
        'previous_module_outcome': previous_module_outcome

    }
    prompt = agent_prompt.format(**prompt_input)
    response = vision.visionModule(query='', prompt=prompt)
    
    response = findAndParseJsonLikeText(response)[0]
    print(f'{response=}')

    module_input = ''
    for module in ['update_goal', 'ask_memories', 'perform_action', 'perceive']:
        if response[module]:
            module_input = module
            print(f'{module_input=}')
            break

    if 'update_goal' in module_input:
        
        # summarize key lessons learned from this goal, save them to memory
        if current_goal:
            key_lessons = act.save_key_lessons(goal=current_goal)
        else:
            key_lessons = 'Just spawned. No key lessons learned.'

        # update the goal and plan
        current_goal, plan = goal.update_goal(bot, key_lessons, response['update_goal'])

        # reset the previous action outcomes and current goal progress
        previous_action_outcomes, current_goal_progress = [], ''

        # outcome of this action is telling the bot that the new goal and plan are set
        outcome = 'New goal and plan have been set.'

    if 'ask_memories' in module_input:
        # ask the memory module for the answer to the question
        question = response['ask_memories']
        answer = mem.query_memories(question)

        # outcome of this action is the answer to the question
        outcome = 'Question: ' + question + '\nAnswer:' + answer

    if 'perform_action' in module_input:
        # attempt to perform the requested action
        action = response['perform_action']
        previous_action_outcomes, current_goal_progress = act.perform_action(bot, action)

        # outcome of this action is an update to the goals and plans
        outcome = 'Action: ' + action + '\nOutcome: ' + current_goal_progress

    if 'perceive' in module_input:
        # perceive the world
        perception = percept.perceive(bot, response['perceive'])

        # outcome of this action is the perception of the world
        outcome = 'Task: ' + response['perceive'] + 'Response:' + perception


    # send the outcome of the action to the agent
    previous_module_outcome = 'Module Requested: ' + str(response) + '\nOutcome: ' + str(outcome)